# Categorical Variables

Intakes:

    Indigenous land from FUNAI
        https://www.gov.br/funai/pt-br/atuacao/terras-indigenas/geoprocessamento-e-mapas
    Ecoregion from RESOLVE
        https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017
    Protected areas from CEM - USP (2020)
        https://centrodametropole.fflch.usp.br/pt-br/download-de-dados
    Biome data from IBGE
        https://www.ibge.gov.br/geociencias/informacoes-ambientais/vegetacao/15842-biomas.html

        For specific biomes, the feature "CD_Bioma" is:
        1 = Amazonia
        2 = Caatinga
        3 = Cerrado
        4 = Mata Atlantica
        5 = Pampa
        6 = Pantanal
    
Exports images with binary masks for protected areas and indigenous land, or byte values for ecoregion and soil type.

In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

In [13]:
biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
biome_img = biome.reduceToImage(["CD_Bioma"], ee.Reducer.first()).byte().rename("biome")

indig_land = ee.FeatureCollection(f"{data_folder}/raw/indig_land").filterBounds(roi)
indig_land_img = ee.Image().paint(indig_land, 1).unmask(0).byte().rename("indig")

protec = ee.FeatureCollection(f"{data_folder}/raw/protec_2021").filterBounds(roi)
protec_img = ee.Image().paint(protec, 1).unmask(0).byte().rename("protec")

categorical = biome_img.addBands([indig_land_img, protec_img])

export_image(categorical, "categorical", region = roi, scale = 500)

## Create distance_to_border mask

Create a mask to keep only pixels at least 10km away from the border of the biomes (national border and border with other biomes). Pixels within 10km of the coastline are not removed.


In [ ]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')

south_american_countries = [
    'Argentina', 'Bolivia', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))

sa = ee.Image.constant(1).clip(south_america).reproject(crs = 'EPSG:4326', scale = 5000)

# Select only the Amazon and Atlantic Forests as desired biomas
biome_mask = (categorical.select('biome').eq(1)
                .Or(categorical.select('biome').eq(4))
                .Or(categorical.select('biome').eq(6))
                .reproject(crs = 'EPSG:4326', scale = 5000))

# Highlight only areas that are not the desired biomes OR oceans
new_sa = (ee.Image(-1).where(sa.mask().Not(), 2)
                      .where(biome_mask.gt(0), 1)
                      .where(biome_mask.eq(0), -1))

edge_detec = new_sa.zeroCrossing() # Detect borders

# measure distance from each pixel to the nearest border
distance_to_border = (edge_detec.fastDistanceTransform().sqrt()
                        .multiply(ee.Image.pixelArea().sqrt())
                        .updateMask(biome_mask)
                        .rename("distance_to_edge"))

distance_to_border_mask = distance_to_border.gt(10000).byte()

export_image(distance_to_border_mask, "distance_to_border_mask", scale = 1000)